In [1]:
import numpy as np

def load_npy_file(file_path):
    return np.load(file_path)[0]

def save_npy_file(data, file_path):
    np.save(file_path, [data])

def trim(data, start_frame, end_frame):
    return data[start_frame:end_frame + 1]

def in_place(data):
    joint_0_position = data[0, 0, :]
    data_adjusted = data - joint_0_position
    for frame in range(len(data_adjusted)):
        data_adjusted[frame, :, :] = data[frame, :, :] - (data[frame, 0, :] - joint_0_position)
        data_adjusted[:, 0, :] = joint_0_position

    return data_adjusted

def increase_smoothness(data, window_size=3):
    frames, joints, axes = data.shape
    smoothed_data = np.copy(data)

    for joint in range(joints):
        for axis in range(axes):
            smoothed_data[:, joint, axis] = np.convolve(data[:, joint, axis], np.ones(window_size) / window_size, mode='same')

    return smoothed_data

def decrease_smoothness(data, noise_level=0.001):
    noise = np.random.normal(0, noise_level, data.shape)
    noisy_data = data + noise
    return noisy_data

def increase_speed(data, factor=2):
    frames, joints, axes = data.shape
    new_frames = frames // factor
    sped_up_data = np.zeros((new_frames, joints, axes))

    for i in range(new_frames):
        sped_up_data[i] = data[i * factor]

    return sped_up_data

def decrease_speed(data, factor=2):
    frames, joints, axes = data.shape
    new_frames = frames * factor
    slowed_down_data = np.zeros((new_frames, joints, axes))

    for i in range(frames):
        slowed_down_data[i * factor] = data[i]

    # Interpolate between repeated frames
    for i in range(1, factor):
        for j in range(frames - 1):
            slowed_down_data[j * factor + i] = (
                slowed_down_data[j * factor] * (factor - i) +
                slowed_down_data[(j + 1) * factor] * i
            ) / factor

    return slowed_down_data

def mirror(data, axis=0):
    mirrored_data = np.copy(data)
    mirrored_data[..., axis] = -mirrored_data[..., axis]
    return mirrored_data

def transition_with_pose(data, pose, transition_frames=10, preserve_pairs=[]):
    frames, joints, axes = data.shape
    start_frame = data[0]
    end_frame = data[-1]
    transition_start = np.zeros((transition_frames, joints, axes))
    transition_end = np.zeros((transition_frames, joints, axes))

    for i in range(transition_frames):
        alpha = i / (transition_frames - 1)
        transition_start_frame = (1 - alpha) * pose + alpha * start_frame
        transition_end_frame = (1 - alpha) * end_frame + alpha * pose
        
        for (joint_a, joint_b) in preserve_pairs:
            original_distance = np.linalg.norm(pose[joint_a] - pose[joint_b])
            
            # For start transition
            current_distance_start = np.linalg.norm(transition_start_frame[joint_a] - transition_start_frame[joint_b])
            if current_distance_start != 0:  # Avoid division by zero
                adjustment_ratio_start = original_distance / current_distance_start
                transition_start_frame[joint_b] = transition_start_frame[joint_a] + (transition_start_frame[joint_b] - transition_start_frame[joint_a]) * adjustment_ratio_start
            
            # For end transition
            current_distance_end = np.linalg.norm(transition_end_frame[joint_a] - transition_end_frame[joint_b])
            if current_distance_end != 0:  # Avoid division by zero
                adjustment_ratio_end = original_distance / current_distance_end
                transition_end_frame[joint_b] = transition_end_frame[joint_a] + (transition_end_frame[joint_b] - transition_end_frame[joint_a]) * adjustment_ratio_end
        
        transition_start[i] = transition_start_frame[1]
        transition_end[i] = transition_end_frame[1]

    data_with_transition = np.concatenate((transition_start, data, transition_end), axis=0)
    return data_with_transition

def end_with_start(data, transition_frames=10, preserve_pairs=[]):
    frames, joints, axes = data.shape
    start_frame = data[0]
    end_frame = data[-1]
    transition = np.zeros((transition_frames, joints, axes))

    for i in range(transition_frames):
        alpha = i / (transition_frames - 1)
        transition_frame = (1 - alpha) * end_frame + alpha * start_frame
        
        for (joint_a, joint_b) in preserve_pairs:
            original_distance = np.linalg.norm(start_frame[joint_a] - start_frame[joint_b])
            current_distance = np.linalg.norm(transition_frame[joint_a] - transition_frame[joint_b])
            adjustment_ratio = original_distance / current_distance
            transition_frame[joint_b] = transition_frame[joint_a] + (transition_frame[joint_b] - transition_frame[joint_a]) * adjustment_ratio
        
        transition[i] = transition_frame

    data_extended = np.concatenate((data, transition), axis=0)
    return data_extended
# Load data
data = load_npy_file('sit.npy')

# Load A pose and T pose
A_pose = load_npy_file('Apose.npy')
T_pose = load_npy_file('Tpose.npy')

# Trim data
start_frame = 10
end_frame = 50
data_trimmed = trim(data, start_frame, end_frame)
save_npy_file(data_trimmed, 'data_trimmed.npy')

# Adjust data to in-place coordinates
data_in_place = in_place(data)
save_npy_file(data_in_place, 'data_in_place.npy')

# # Increase smoothness
data_smooth = increase_smoothness(data_in_place, window_size=3)
save_npy_file(data_smooth, 'data_smooth.npy')

# # Decrease smoothness
data_noisy = decrease_smoothness(data_in_place, noise_level=0.003)
save_npy_file(data_noisy, 'data_noisy.npy')

# # Increase speed
data_sped_up = increase_speed(data_in_place, factor=2)
save_npy_file(data_sped_up, 'data_sped_up.npy')

# # Decrease speed
data_slowed_down = decrease_speed(data_in_place, factor=2)
save_npy_file(data_slowed_down, 'data_slowed_down.npy')

# # Mirror data
data_mirrored = mirror(data_in_place, axis=0)
save_npy_file(data_mirrored, 'data_mirrored.npy')

# Ensure animation starts and ends with the A pose, preserving distances between specified joints
preserve_pairs = [(0, 1), (0, 2), (2, 5), (5, 8), (8, 11), (1, 4), (4, 7), (7, 10), (0, 3), (3, 6), (6, 9), (9, 12), (9, 13), (9, 14), (14, 17), (17, 19), (19, 21), (13, 16), (16, 18), (18, 20), (12, 15)]  # Example pairs of joints to preserve distances
data_with_A_pose = transition_with_pose(data, A_pose, transition_frames=10, preserve_pairs=preserve_pairs)
save_npy_file(data_with_A_pose, 'data_with_A_pose.npy')

# Ensure animation starts and ends with the T pose, preserving distances between specified joints
data_with_T_pose = transition_with_pose(data, T_pose, transition_frames=10, preserve_pairs=preserve_pairs)
save_npy_file(data_with_T_pose, 'data_with_T_pose.npy')

data_ended_with_start = end_with_start(data, transition_frames=10, preserve_pairs=preserve_pairs)
save_npy_file(data_ended_with_start, 'data_ended_with_start.npy')
